In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
import re
import warnings 
warnings.filterwarnings('ignore') 

Introdruction:  
Partner: Roxylynn Roach  

In California as well as many other states, our schools are funded in large part by local property taxes of the surrounding neighborhoods. Schools in affluent neighborhoods receive a larger amount of funding than those of struggling neighborhoods. This difference in funding creates a disparity in the quality of education our children receive with some receiving good quality education and others receiving very poor education. This problem is heightened by the personal circumstances of our students. Students from poorer neighborhoods will not be prioritizing their education because they are more focused on working to create a supplemental income for their family to survive on. Others may simply be unmotivated to learn due to the quality of their education. Regardless, we want to help identify these struggling communities with struggling students. If we are able to do so, we hope that we may be able to provide much needed financial assistance to these communities and these schools.  

To do so we are going to compare CAASPP ELA testing scores with average income by county. If we are successfully able to find a correlation between the two, then we will have successfully found a predictor to find our most struggling communities. Of course, economic hardships will not be the only cause for low test scores, but we hope that average income will have a strong enough correlation so that we can help as many people as possible.  


Using CAASP ELA data rather then ELPAC. CAASP measures english language arts and mathematics and ELPAC is a english language test for students whose first language is a language other then English. CAASP will be good for broad stroke analysis of literecy and we can use ELPAC as a second exploritory variable to explore english learners and compare them to CAASP ELA.

CAASP ELA source: https://caaspp-elpac.ets.org/caaspp/ResearchFileListSB  
Population census source: https://labormarketinfo.edd.ca.gov/Population_and_Census.html#PRO  
Income census source: https://labormarketinfo.edd.ca.gov/Population_and_Census.html#POP  

Starting by assigning the base population and income data sets.  
Next I will check for duplicate and null values.

In [2]:
population_data = pd.read_csv('population_data.csv')

In [3]:
population_data

,Year,Period,Area,Population Source,Population
0,2022,Annual,California,California Dept of Finance,"39,146,273"
1,2021,Annual,California,California Dept of Finance,"39,246,702"
2,2020,Annual,California,California Dept of Finance,"39,541,722"
3,2019,Annual,California,California Dept of Finance,"39,529,566"
4,2018,Annual,California,California Dept of Finance,"39,476,064"
...,...,...,...,...,...
526,2018,Annual,Yuba County,California Dept of Finance,"79,043"
527,2017,Annual,Yuba County,California Dept of Finance,"78,181"
528,2016,Annual,Yuba County,California Dept of Finance,"76,782"
529,2015,Annual,Yuba County,California Dept of Finance,"75,748"


In [4]:
population_data.isna().sum()

Year                 0
Period               0
Area                 0
Population Source    0
Population           0
dtype: int64

In [5]:
population_data.duplicated().sum()

0

In [6]:
income_data = pd.read_csv('income_data.csv')

In [7]:
income_data

,Year,Period,Area,Income Source,Income Type,Income
0,2022,Annual,California,U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$77,036"
1,2022,Annual,California,U.S. BEA - Local Data,Total Personal Income - BEA,"$3,006,647,281"
2,2021,Annual,California,U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$76,991"
3,2021,Annual,California,U.S. BEA - Local Data,Total Personal Income - BEA,"$3,013,676,929"
4,2020,Annual,California,U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$70,061"
...,...,...,...,...,...,...
1057,2016,Annual,Yuba County,U.S. BEA - Local Data,Total Personal Income - BEA,"$2,853,823"
1058,2015,Annual,Yuba County,U.S. BEA - Local Data,Total Personal Income - BEA,"$2,738,968"
1059,2015,Annual,Yuba County,U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$36,587"
1060,2014,Annual,Yuba County,U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$20,471"


In [8]:
income_data.isna().sum()

Year             0
Period           0
Area             0
Income Source    0
Income Type      0
Income           0
dtype: int64

In [9]:
income_data.duplicated().sum()

0

Now I'm going to merge and clean my data sets. Because the income data comes in 2 types, total personal income, and per capita personal income, I am going to create 2 different data sets.  
First, divide my data sets on those income types  
Second, check the number of rows. If done correctly, both should have 531 rows  
Third, merge data frames  
Fourth, trim data frames
Fith, double check merged data frames for row numbers, null values, and duplicates.

In [10]:
tpi_type = income_data[income_data['Income Type'] == 'Total Personal Income - BEA']
len(tpi_type)

531

In [11]:
pcpi_type = income_data[income_data['Income Type'] == 'Per Capita Personal Income - BEA']
len(pcpi_type)

531

In [12]:
total_personal_income_raw = pd.merge(population_data, tpi_type, on = ['Year', 'Period', 'Area'], how = 'inner')
total_personal_income_raw

,Year,Period,Area,Population Source,Population,Income Source,Income Type,Income
0,2022,Annual,California,California Dept of Finance,"39,146,273",U.S. BEA - Local Data,Total Personal Income - BEA,"$3,006,647,281"
1,2021,Annual,California,California Dept of Finance,"39,246,702",U.S. BEA - Local Data,Total Personal Income - BEA,"$3,013,676,929"
2,2020,Annual,California,California Dept of Finance,"39,541,722",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,767,521,379"
3,2019,Annual,California,California Dept of Finance,"39,529,566",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,537,950,599"
4,2018,Annual,California,California Dept of Finance,"39,476,064",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,411,055,136"
...,...,...,...,...,...,...,...,...
526,2018,Annual,Yuba County,California Dept of Finance,"79,043",U.S. BEA - Local Data,Total Personal Income - BEA,"$3,108,945"
527,2017,Annual,Yuba County,California Dept of Finance,"78,181",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,937,225"
528,2016,Annual,Yuba County,California Dept of Finance,"76,782",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,853,823"
529,2015,Annual,Yuba County,California Dept of Finance,"75,748",U.S. BEA - Local Data,Total Personal Income - BEA,"$2,738,968"


In [13]:
per_capita_income_raw = pd.merge(population_data, pcpi_type, on = ['Year', 'Period', 'Area'], how = 'inner')
per_capita_income_raw

,Year,Period,Area,Population Source,Population,Income Source,Income Type,Income
0,2022,Annual,California,California Dept of Finance,"39,146,273",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$77,036"
1,2021,Annual,California,California Dept of Finance,"39,246,702",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$76,991"
2,2020,Annual,California,California Dept of Finance,"39,541,722",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$70,061"
3,2019,Annual,California,California Dept of Finance,"39,529,566",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$64,174"
4,2018,Annual,California,California Dept of Finance,"39,476,064",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$60,984"
...,...,...,...,...,...,...,...,...
526,2018,Annual,Yuba County,California Dept of Finance,"79,043",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$39,466"
527,2017,Annual,Yuba County,California Dept of Finance,"78,181",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$37,776"
528,2016,Annual,Yuba County,California Dept of Finance,"76,782",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$37,593"
529,2015,Annual,Yuba County,California Dept of Finance,"75,748",U.S. BEA - Local Data,Per Capita Personal Income - BEA,"$36,587"


Time to trim these data frame columns. I'm going to get rid of the period, source information, and income type.

In [14]:
total_personal_income = total_personal_income_raw.drop(columns=
                                                       ['Period', 'Population Source', 'Income Source', 'Income Type'])
total_personal_income

,Year,Area,Population,Income
0,2022,California,"39,146,273","$3,006,647,281"
1,2021,California,"39,246,702","$3,013,676,929"
2,2020,California,"39,541,722","$2,767,521,379"
3,2019,California,"39,529,566","$2,537,950,599"
4,2018,California,"39,476,064","$2,411,055,136"
...,...,...,...,...
526,2018,Yuba County,"79,043","$3,108,945"
527,2017,Yuba County,"78,181","$2,937,225"
528,2016,Yuba County,"76,782","$2,853,823"
529,2015,Yuba County,"75,748","$2,738,968"


In [15]:
per_capita_income = per_capita_income_raw.drop(columns= 
                                               ['Period', 'Population Source', 'Income Source', 'Income Type'])
per_capita_income

,Year,Area,Population,Income
0,2022,California,"39,146,273","$77,036"
1,2021,California,"39,246,702","$76,991"
2,2020,California,"39,541,722","$70,061"
3,2019,California,"39,529,566","$64,174"
4,2018,California,"39,476,064","$60,984"
...,...,...,...,...
526,2018,Yuba County,"79,043","$39,466"
527,2017,Yuba County,"78,181","$37,776"
528,2016,Yuba County,"76,782","$37,593"
529,2015,Yuba County,"75,748","$36,587"


Double check trimmed data frames for row numbers, null values, and duplicates.

In [16]:
total_personal_income.isna().sum()

Year          0
Area          0
Population    0
Income        0
dtype: int64

In [17]:
total_personal_income.duplicated().sum()

0

In [18]:
per_capita_income.isna().sum()

Year          0
Area          0
Population    0
Income        0
dtype: int64

In [19]:
per_capita_income.duplicated().sum()

0

Everything looks good here.  
I might want more demographic info like ethnicity but for now lets move on.

So this is the tough part. I have a lot of CAASPP data and I'm not entirely sure how I want to handle it. I could merge it all but I think the total data frame will be too big to work with efficently. Let's explore the first year and see if we can work it out.  

In [20]:
casp_2015_raw = pd.read_csv('2014-2015.csv')
casp_2015_raw

,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total CAASPP Enrollment,Total Tested At Entity Level,Total Tested at Subgroup Level,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage At or Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage At or Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage At or Near Standard,Area 4 Percentage Below Standard
0,0,0,0,NaN,2015,1,B,3824705,3684180,3684180,...,40,20,42,38,20,52,28,0,0,0
1,0,0,0,NaN,2015,1,B,3824705,3684180,3684180,...,41,17,44,38,14,62,24,18,50,32
2,0,0,0,NaN,2015,1,B,3824705,3684180,3684180,...,39,18,47,35,15,62,23,18,51,31
3,0,0,0,NaN,2015,1,B,3824705,3684180,3684180,...,47,16,45,39,18,43,39,0,0,0
4,0,0,0,NaN,2015,1,B,3824705,3684180,3684180,...,51,16,38,46,14,45,41,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106266,58,72769,123570,NaN,2015,1,B,6,6,6,...,*,*,*,*,*,*,*,*,*,*
106267,58,72769,5838305,NaN,2015,1,B,348,336,336,...,45,7,66,27,8,60,32,0,0,0
106268,58,72769,5838305,NaN,2015,1,B,348,336,336,...,11,33,53,14,23,64,14,41,54,5
106269,58,72769,5838305,NaN,2015,1,B,348,336,336,...,11,33,53,14,23,64,14,41,54,5


Yeah that's a big data frame. Each row is one student so each year will have a different number of students but we can expect them to be comparable to this one at ~100000 students. Lets look at the variables.

https://caaspp-elpac.ets.org/caaspp/ResearchFileFormatSB?ps=true&lstTestYear=2015
County code - county code  
District code - district code  
School code - school code  
Filler - empty dummy variable  
Test Year - year the test was taken, tests are taken in spring so if test year is 2015, the school year is 2014-2015   
Subgroup ID - Student demographics, this data set only has '1' which represents all students  
Test Type - B = English test or math test. A = math  
Total CAASPP enrollment - total number of students who were suppose to take the CAASPP test  
Total tested at entity level - number of students who actually took the test  
total tested at subgroup level - Total Tested with Scores  
grade - grade level of student 3-11, 13 represents all grades  
test id - 1 for english 2 for math  
caaspp reported enrollment - number of students caaspp reports having tested  
students tested - actual number of students tested  
mean scale score - mean score ranging from 2000 to 3000  
percentage standard exceeded - percentage of students who achieved score of 'exceeded expectations'  
percentage standard met - percentage of students who achieved score of 'met expectations'  
percentage standard met and above - percentage of students who achieved score of 'met and are slightly above expectations'  
percentage standard nearly met - percentage of students who nearly met score expectations  
percentage standard not met - percentage of students who did not meet expectations  
students with scores - number of students with scores  
Area 1 percentage above standard - percentage of students who achieved above standard in area 1 of the exam  
area 1 Percentage At or Near Standard - percentage of students who achieved at or near standard in area 1 of the exam  
Area 1 Percentage Below Standard - percentage of students who achieved below standard in area 1 of the exam  
Area 2 Percentage Above Standard - percentage of students who achieved above standard in area 2 of the exam  
Area 2 Percentage At or Near Standard - percentage of students who achieved at or near standard in area 2 of the exam  
Area 2 Percentage Below Standard - percentage of students who achieved below standard in area 2 of the exam  
Area 3 Percentage Above Standard - percentage of students who achieved above standard in area 3 of the exam  
Area 3 Percentage At or Near Standard - percentage of students who achieved at or near standard in area 3 of the exam  
Area 3 Percentage Below Standard - percentage of students who achieved below standard in area 3 of the exam  
Area 4 Percentage Above Standard - percentage of students who achieved above standard in area 4 of the exam  
Area 4 Percentage At or Near Standard - percentage of students who achieved at or near standard in area 4 of the exam  
Area 4 Percentage Below Standard - percentage of students who achieved below standard in area 4 of the exam  

Yeah LOTS of columns, some of them are redundent because the data set is already filtered to the english portion of the test so we can drop those columns. And there are a couple others that we won't be needing. Some I don't think I'll need but they look interesting so I'll keep them for now. If I don't want them I can come back here and drop them as well.

In [21]:
casp_2015 = casp_2015_raw.drop(columns = ['School Code', 'Filler', 'Subgroup ID', 'Test Type', 'Total CAASPP Enrollment', 
                                          'Total Tested At Entity Level', 'Total Tested at Subgroup Level', 'Test Id', 
                                          'CAASPP Reported Enrollment'])
casp_2015

,County Code,District Code,Test Year,Grade,Students Tested,Mean Scale Score,Percentage Standard Exceeded,Percentage Standard Met,Percentage Standard Met and Above,Percentage Standard Nearly Met,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage At or Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage At or Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage At or Near Standard,Area 4 Percentage Below Standard
0,0,0,2015,3,477045,2415.1,14,26,40,27,...,40,20,42,38,20,52,28,0,0,0
1,0,0,2015,3,475228,2402.9,18,20,38,26,...,41,17,44,38,14,62,24,18,50,32
2,0,0,2015,4,465425,2445.7,19,21,40,21,...,39,18,47,35,15,62,23,18,51,31
3,0,0,2015,4,466980,2453.7,13,22,35,35,...,47,16,45,39,18,43,39,0,0,0
4,0,0,2015,5,464153,2480.3,15,15,30,29,...,51,16,38,46,14,45,41,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106266,58,72769,2015,13,5,NaN,*,*,*,*,...,*,*,*,*,*,*,*,*,*,*
106267,58,72769,2015,11,148,2559.7,4,18,22,38,...,45,7,66,27,8,60,32,0,0,0
106268,58,72769,2015,11,150,2619.5,27,40,67,26,...,11,33,53,14,23,64,14,41,54,5
106269,58,72769,2015,13,150,NaN,27,40,67,26,...,11,33,53,14,23,64,14,41,54,5


Now let's check for any null rows and check for duplicates.

In [22]:
casp_2015.isna().sum()

County Code                                  0
District Code                                0
Test Year                                    0
Grade                                        0
Students Tested                              0
Mean Scale Score                         24973
Percentage Standard Exceeded                 0
Percentage Standard Met                      0
Percentage Standard Met and Above            0
Percentage Standard Nearly Met               0
Percentage Standard Not Met                  0
Students with Scores                         0
Area 1 Percentage Above Standard             0
Area 1 Percentage At or Near Standard        0
Area 1 Percentage Below Standard             0
Area 2 Percentage Above Standard             0
Area 2 Percentage At or Near Standard        0
Area 2 Percentage Below Standard             0
Area 3 Percentage Above Standard             0
Area 3 Percentage At or Near Standard        0
Area 3 Percentage Below Standard             0
Area 4 Percen

25000 null values for mean scale score. After looking at it closer, we can see that the grade level for these null values is '13'. This represents the commulative 

In [23]:
casp_2016_raw = pd.read_csv('2015-2016.csv')
casp_2016_raw

,County Code,District Code,School Code,Filler,Test Year,Subgroup ID,Test Type,Total CAASPP Enrollment,Total Tested At Entity Level,Total Tested with Scores,...,Area 1 Percentage Below Standard,Area 2 Percentage Above Standard,Area 2 Percentage Near Standard,Area 2 Percentage Below Standard,Area 3 Percentage Above Standard,Area 3 Percentage Near Standard,Area 3 Percentage Below Standard,Area 4 Percentage Above Standard,Area 4 Percentage Near Standard,Area 4 Percentage Below Standard
0,0,0,0,NaN,2016,1,B,3329946,3210627,3196619,...,37,21,45,34,17,63,20,22,49,29
1,0,0,0,NaN,2016,1,B,3329889,3220980,3204822,...,35,23,45,32,25,53,22,0,0,0
2,0,0,0,NaN,2016,1,B,3329946,3210627,3196619,...,37,22,46,32,16,65,19,22,50,28
3,0,0,0,NaN,2016,1,B,3329889,3220980,3204822,...,44,18,46,36,21,45,34,0,0,0
4,0,0,0,NaN,2016,1,B,3329946,3210627,3196619,...,36,27,42,31,17,61,22,31,51,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98708,58,72769,123570,NaN,2016,1,B,1,1,1,...,*,*,*,*,*,*,*,*,*,*
98709,58,72769,5838305,NaN,2016,1,B,157,153,152,...,60,5,57,38,11,62,28,0,0,0
98710,58,72769,5838305,NaN,2016,1,B,157,152,152,...,18,30,55,15,16,73,11,35,55,11
98711,58,72769,5838305,NaN,2016,1,B,157,153,152,...,60,5,57,38,11,62,28,0,0,0


In [24]:
casp_2015.dtypes

County Code                               int64
District Code                             int64
Test Year                                 int64
Grade                                     int64
Students Tested                           int64
Mean Scale Score                         object
Percentage Standard Exceeded             object
Percentage Standard Met                  object
Percentage Standard Met and Above        object
Percentage Standard Nearly Met           object
Percentage Standard Not Met              object
Students with Scores                      int64
Area 1 Percentage Above Standard         object
Area 1 Percentage At or Near Standard    object
Area 1 Percentage Below Standard         object
Area 2 Percentage Above Standard         object
Area 2 Percentage At or Near Standard    object
Area 2 Percentage Below Standard         object
Area 3 Percentage Above Standard         object
Area 3 Percentage At or Near Standard    object
Area 3 Percentage Below Standard        

Goals moving forward:  
- function for cleaning caaspp data sets
- seperate null value rows into a different data frame
- identify lowest, medium, and highest average income counties.
    - random sample a number of students for these counties for each year. This is the data we will use for visualization  